In [1]:
import spotipy,os
import spotipy.util as util
import json
import requests as rq

client_id = 'a5f4717d6cbe43fea2cc354da04490b6'
redirect_uri = 'http://localhost:8888/callback/'
client_secret = '4eaba7b3f548455dae6a2ac83d8e39e1'
scope = ['user-library-read','user-read-private','user-read-recently-played','user-read-currently-playing']


Oauth = spotipy.oauth2.SpotifyOAuth(client_id=client_id,
                            client_secret=client_secret,
                            redirect_uri=redirect_uri,
                            scope=scope)
token = Oauth.get_access_token()
print(token)
headers = {"Authorization": "Bearer {}".format(token['access_token'])}

sp = spotipy.client.Spotify(headers)
data = sp.current_user_recently_played(50)

items =  data['items']
# print(items)
# artist = sp.artist("2yRnjWtHzmDELwYaUiX0Yh")
# print(artist)
#get artist uri
# d = items[0]['track']
# print(d['artists'][0]['uri'])
# uri = d['artists'][0]['uri']
# id = uri.split(":", 2)[2]
# print(id)
# URL = 'https://api.spotify.com/v1/artists/{}'.format(id[2])


{'access_token': 'BQBfWrrRz-VveEw0P9T7tZVZoQIu6y0kxkrfkyZM0m9IMgljOVwgF4qzNLEkTGlIUXCxiib6ufE0FUjeCVubSc6c2UdfEPfMUqG7x4icQqAxW844FPkr2x0JnLNSX1Iu6xDEeUtfIJgAEJ_PJboVDR3LgcfGQUqSN9Ic3xDTAOfoJlKuczwRjXJDkSrfXYP-Gq0cWvuH1Lie3-kmiek', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-library-read user-read-currently-playing user-read-private user-read-recently-played', 'expires_at': 1708255615, 'refresh_token': 'AQDROefR5cOiwNcyosQ5lSrWWm1mSOTpAMYEueWCc4Sm4vwphUW0P6oNQQlNSf36jp5NytzbAfN_tjRqji3Ik3FB493fTqXUaoar1pUmqEpOiwupj7aS_PUmC-KFu2oIQSM'}


C:\Users\kevin\AppData\Local\Temp\ipykernel_27804\4036136410.py:16: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = Oauth.get_access_token()


In [18]:
def get_artist_genres(artist_id):
    URL = 'https://api.spotify.com/v1/artists/{}'.format(artist_id)
    output = rq.get(URL,headers=headers).json()

    return output['genres']

data  = find_songs_name(items)
# print(data)
for song in data[:]:
    song[-1] = get_artist_genres(song[-1])
    
print(data)


[['勇者', 'YOASOBI', 'https://i.scdn.co/image/ab67616d00001e02a9f9b6f07b43009f5b0216dc', '2024-02-18 08:16:30', '64tJ2EAv1R6UaZqc4iOCyj'], ['more than words', 'Hitsujibungaku', 'https://i.scdn.co/image/ab67616d00001e0209fdd610f8a7b39b3b7260ef', '2024-02-18 08:16:19', '6S8w5rLsEwjN21jQeRES0n'], ['アンビバレント', 'Uru', 'https://i.scdn.co/image/ab67616d00001e025f695a1401f9626e21692ecb', '2024-02-18 08:11:28', '6le80R1opKnTlP4o3KvA2k'], ['「僕は...」', 'あたらよ', 'https://i.scdn.co/image/ab67616d00001e02d230bbe17b4e2f279e42c1fe', '2024-02-18 08:07:26', '2yRnjWtHzmDELwYaUiX0Yh'], ['SPECIALZ', 'King Gnu', 'https://i.scdn.co/image/ab67616d00001e0288f868f81564eaa604ec08ef', '2024-02-18 08:03:26', '6wxfx1yhyqjCPYwwxJktR2'], ['憂い桜', 'あたらよ', 'https://i.scdn.co/image/ab67616d00001e029efd9858adaf5e9af4c817c9', '2024-02-18 07:59:25', '2yRnjWtHzmDELwYaUiX0Yh'], ['また夏を追う', 'あたらよ', 'https://i.scdn.co/image/ab67616d00001e029efd9858adaf5e9af4c817c9', '2024-02-18 07:58:15', '2yRnjWtHzmDELwYaUiX0Yh'], ['僕らはそれを愛と呼んだ', 'あ

In [2]:
def find_songs_name(items):
    last50songs = []
    for i in range(len(items)):
        data = []
        current_track = items[i]['track']
        time = items[i]['played_at']
        album = current_track['album']
        album_name = album['name']
        album_image_inf = album['images'][1]
        album_image_url = album_image_inf['url']
        artist_inf = album['artists']
        artist_name = artist_inf[0]['name']
        artist_uri = artist_inf[0]['uri'].split(":", 2)[2]

        t = trans_time(time)
        # print(str(t))

        data.append(current_track['name'])
        data.append(artist_name)
        data.append(album_image_url)
        data.append(t)
        data.append(artist_uri)
        last50songs.append(data)

    return last50songs
    
def take_second(list):
    return list[2]

def trans_time(t):
    t = t.split('T')
    date = t[0]
    time = t[1].split('.')[0]
    return date+" "+time

def get_data():

    token = get_token()
    headers = {"Authorization": "Bearer {}".format(token['access_token'])}

    sp = spotipy.client.Spotify(headers)
    data = sp.current_user_recently_played(50)

    items =  data['items']

    last50songs = find_songs_name(items)

    return last50songs

def count_song(data):
    song_count = {}
    song_list = []
    for song in data:
        # print(song)
        song_name = song[0]
        artist_name =  song[1]
        song_image_url = song[2]
        if song_name in song_count:
            continue
        else:
            song_count.setdefault(song_name,[])
            song_count[song_name].append(0)
            song_count[song_name].append(artist_name)
            song_count[song_name].append(song_image_url)

    for song in data:
        song_name = song[0]
        song_count[song_name][0] +=1

    for item in song_count:
        dic = {}
        dic['SongName'] = item
        dic['Artist'] = song_count[item][1]
        dic['Cover'] = song_count[item][2]
        dic['Count'] = song_count[item][0]
        data = [item, song_count[item][1] ,song_count[item][2],song_count[item][0]]
        song_list.append(data)

    song_list.sort(key=take_second,reverse=True)
    
    return song_list


def get_user_info():
    token = get_token()
    headers = {"Authorization": "Bearer {}".format(token['access_token'])}
    sp = spotipy.client.Spotify(headers)
    user = sp.me()
    l = []
    l.append(user['display_name'])
    l.append(user['images'][0]['url'] )

    return l


In [12]:
import psycopg2
song_detail = ['僕らはそれを愛と呼ん', 'あたらよ', 'https://i.scdn.co/image/ab67616d00001e029efd9858adaf5e9af4c817c9', '2024-02-17 15:54:16']

connection = psycopg2.connect(database="postgres",user="postgres",password="postgres",host="127.0.0.1",port='5432')

cursor = connection.cursor()

# for i in data[:1]:
#     song_detail = i[:4]
#     genre_data = i[4]
#     # print(type(song_detail[1]))
#     # print(song_detail)
cursor.execute("INSERT INTO listened_list (song_name, artist, image_url, timestamp_column) VALUES(%s, %s, %s, %s) RETURNING song_id", song_detail)
print(cursor.fetchone()[0])
connection.commit()

# print(song_id)
# cursor.execute("SELECT * FROM listened_list")
# print(cursor.fetchone()[0])
# print(cursor.statusmessage)



1


In [24]:
connection = psycopg2.connect(database="postgres",user="postgres",password="postgres",host="127.0.0.1",port='5432')

cursor = connection.cursor()
cursor.execute("SELECT * FROM listened_list")

print(cursor.fetchone()[0])

connection.commit()
connection.close()

9
